In [0]:
import os

import numpy as np
from matplotlib import pyplot as plt
import cv2
import xml.etree.ElementTree as ET
from collections import Counter

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

#run locally as it trains in seconds and gdrive is slow with many files
PATH = './'

In [0]:
X = np.zeros((3240,20,20), dtype=float)
y = np.zeros(3240, dtype=float)

In [0]:
for i, file in enumerate(os.listdir(PATH+'segments/')):
        img = cv2.imread(PATH+'/segments/'+file, cv2.IMREAD_GRAYSCALE)
        #print('iteration {}, file {}'.format(i,file))
        X[i,:] = img
        tree = ET.parse(PATH+'segment_labels/'+file.rstrip('png')+'xml')
        root = tree.getroot()
        tag = root.find('object').findtext('name')
        if tag == 'empty':
            y[i] = 0
        elif tag == 'black':
            y[i] = 1
        elif tag == 'white':
            y[i] = 2
        

In [0]:
print(Counter(y))

Counter({0.0: 2230, 2.0: 506, 1.0: 504})


In [0]:
#convert to values between 0 and 1
X = X / 255.0

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    shuffle=True)

In [0]:
keras.backend.clear_session()

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(20, 20)),
    keras.layers.Dense(128, activation=tf.nn.relu, 
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

W0828 11:54:40.798753 4663530944 deprecation.py:506] From /Users/tristan/anaconda3/envs/python3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, 
                    epochs=20, 
                    verbose = 1)

Epoch 1/20
2916/2916 [==============================] - 0s 123us/sample - loss: 1.5835 - acc: 0.8748
Epoch 2/20
2916/2916 [==============================] - 0s 65us/sample - loss: 0.6863 - acc: 0.9671
Epoch 3/20
2916/2916 [==============================] - 0s 63us/sample - loss: 0.3963 - acc: 0.9763
Epoch 4/20
2916/2916 [==============================] - 0s 65us/sample - loss: 0.2578 - acc: 0.9777
Epoch 5/20
2916/2916 [==============================] - 0s 64us/sample - loss: 0.1903 - acc: 0.9811
Epoch 6/20
2916/2916 [==============================] - 0s 65us/sample - loss: 0.1640 - acc: 0.9815
Epoch 7/20
2916/2916 [==============================] - 0s 65us/sample - loss: 0.1186 - acc: 0.9887
Epoch 8/20
2916/2916 [==============================] - 0s 69us/sample - loss: 0.1017 - acc: 0.9897
Epoch 9/20
2916/2916 [==============================] - 0s 68us/sample - loss: 0.0868 - acc: 0.9921
Epoch 10/20
2916/2916 [==============================] - 0s 64us/sample - loss: 0.0846 - acc: 0.989

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

324/324 [==============================] - 0s 254us/sample - loss: 0.0627 - acc: 0.9969
Test accuracy: 0.99691355


In [0]:
keras.models.save_model(
    model,
    filepath='./Stone_Classifier.h5',
    overwrite=True,
    include_optimizer=False
)

In [0]:
def predict_once(img):
    img = (np.expand_dims(img,0))    #keras expects batch shape
    predictions = model.predict(img) 
    result = np.argmax(predictions[0]) #select most likely class
    return result

In [0]:
img = cv2.imread('./segments/50_w_IMG_0713.png', cv2.IMREAD_GRAYSCALE)
img = img / 255
print(img.shape)
print(predict_once(img))

(20, 20)
1
